# 集客率改良（会場別・作り直し版）

このノートブックは `AJ-*.csv` を **1ファイル=1会場** として個別集計します。

- 会場ごとに市区町村別の来場者数・集客率を算出
- 会場ごとの地図（HTML）を出力
- すべての会場の集客率統計量（平均/中央値/四分位など）を算出

> 重要: 市区町村列がないAJファイルでも、`郵便番号` があれば `utf_ken_all.csv` から補完します。


In [ ]:
# Colab想定（ローカルなら必要に応じてコメントアウト）
!pip -q install pandas numpy openpyxl folium chardet


In [ ]:
import glob
import json
import re
from pathlib import Path

import chardet
import folium
import numpy as np
import pandas as pd


In [ ]:
# ===== 設定 =====
VISITOR_GLOB = "AJ-*.csv"
KEN_ALL_CSV = "utf_ken_all.csv"  # 郵便番号→市区町村の補完に使用
POP_XLSX = "【総計】市区町村別年齢階級別人口(2025.8).xlsx"
POP_SHEET = 0
GEOJSON_PATH = "N03-20240101.geojson"
OUT_DIR = Path("out_venue")
OUT_DIR.mkdir(exist_ok=True)

# AJ側の列候補
MUNI_COL_CANDIDATES = ["市区町村", "居住地_市区町村", "住所_市区町村", "市区町村名"]
ZIP_COL_CANDIDATES = ["郵便番号", "郵便", "zip", "Zip", "ZIP"]

# 人口側の列候補（見つからなければ自動推定）
POP_MUNI_COL_CANDIDATES = ["市区町村", "市区町村名", "自治体名"]
POP_TOTAL_COL_CANDIDATES = ["総数", "人口", "総人口", "人口計"]


In [ ]:
def detect_encoding(path, nbytes=200_000):
    with open(path, "rb") as f:
        raw = f.read(nbytes)
    return chardet.detect(raw).get("encoding") or "utf-8"


def read_csv_flex(path):
    tried = []
    for enc in [detect_encoding(path), "cp932", "shift_jis", "utf-8-sig", "utf-8"]:
        if enc in tried:
            continue
        tried.append(enc)
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            pass
    return pd.read_csv(path)


def first_existing_col(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None


def first_keyword_col(df, include_keywords, exclude_keywords=None):
    exclude_keywords = exclude_keywords or []
    for c in df.columns:
        cs = str(c)
        if all(k in cs for k in include_keywords) and not any(x in cs for x in exclude_keywords):
            return c
    return None


def normalize_muni_name(s: pd.Series) -> pd.Series:
    return (
        s.astype(str)
         .str.strip()
         .str.replace(r"\s+", "", regex=True)
         .str.replace("ヶ", "ケ")
    )


def normalize_zip(s: pd.Series) -> pd.Series:
    return (
        s.astype(str)
         .str.replace(r"[^0-9]", "", regex=True)
         .str.zfill(7)
         .str[:7]
    )


def to_numeric_soft(s: pd.Series) -> pd.Series:
    return pd.to_numeric(s.astype(str).str.replace(",", "", regex=False), errors="coerce")


def extract_venue_name(file_path: str) -> str:
    m = re.match(r"AJ-(.*)\.csv$", Path(file_path).name)
    return m.group(1) if m else Path(file_path).stem


In [ ]:
def build_zip_to_muni_map(ken_all_csv: str):
    ken = pd.read_csv(ken_all_csv, header=None, dtype=str, encoding="utf-8-sig")
    zip_s = normalize_zip(ken.iloc[:, 2])
    muni = normalize_muni_name(ken.iloc[:, 6].fillna("") + ken.iloc[:, 7].fillna(""))
    m = pd.DataFrame({"zip": zip_s, "市区町村": muni})
    m = m[(m["zip"].str.len() == 7) & (m["市区町村"] != "")]
    return dict(zip(m["zip"], m["市区町村"]))


def load_population_df(path, sheet=0):
    header_candidates = [0, 1, 2, 3]
    tried_meta = []

    for h in header_candidates:
        try:
            pop = pd.read_excel(path, sheet_name=sheet, header=h)
        except Exception as e:
            tried_meta.append({"header": h, "error": str(e)})
            continue

        pop.columns = [str(c).strip() for c in pop.columns]

        muni_col = first_existing_col(pop, POP_MUNI_COL_CANDIDATES)
        total_col = first_existing_col(pop, POP_TOTAL_COL_CANDIDATES)

        if muni_col is None:
            muni_col = first_keyword_col(pop, ["市区町村"], exclude_keywords=["コード"])

        if total_col is None:
            total_col = (
                first_keyword_col(pop, ["総人口"]) or
                first_keyword_col(pop, ["人口", "総数"]) or
                first_keyword_col(pop, ["人口計"]) or
                first_keyword_col(pop, ["人口"], exclude_keywords=["率", "コード"])
            )

        if muni_col is None:
            object_cols = [c for c in pop.columns if pop[c].dtype == "object"]
            muni_col = object_cols[0] if object_cols else pop.columns[0]

        if total_col is None:
            best_col, best_score = None, -1
            for c in pop.columns:
                if c == muni_col:
                    continue
                score = to_numeric_soft(pop[c]).notna().sum()
                if score > best_score:
                    best_col, best_score = c, score
            total_col = best_col

        if total_col is None:
            tried_meta.append({"header": h, "columns": list(pop.columns), "reason": "total_col_not_found"})
            continue

        out = pop[[muni_col, total_col]].copy()
        out.columns = ["市区町村", "人口"]
        out["市区町村"] = normalize_muni_name(out["市区町村"])
        out["人口"] = to_numeric_soft(out["人口"])
        out = out.dropna(subset=["市区町村", "人口"])
        out = out[out["市区町村"] != ""]

        if len(out) > 0:
            return out.groupby("市区町村", as_index=False)["人口"].sum(), {
                "header": h,
                "muni_col": muni_col,
                "pop_col": total_col,
            }

        tried_meta.append({"header": h, "columns": list(pop.columns), "reason": "empty_after_clean"})

    raise KeyError(f"人口データの列自動判定に失敗: {tried_meta[:3]}")


In [ ]:
# ===== 集計実行 =====
pop_df, pop_meta = load_population_df(POP_XLSX, sheet=POP_SHEET)
print(f"人口列判定: header={pop_meta['header']} / muni={pop_meta['muni_col']} / pop={pop_meta['pop_col']}")

zip_to_muni = build_zip_to_muni_map(KEN_ALL_CSV)
csv_paths = sorted(glob.glob(VISITOR_GLOB))
if not csv_paths:
    raise FileNotFoundError(f"{VISITOR_GLOB} が見つかりません")

records = []
venue_detail_tables = {}

for p in csv_paths:
    venue = extract_venue_name(p)
    df = read_csv_flex(p)

    muni_col = first_existing_col(df, MUNI_COL_CANDIDATES)
    zip_col = first_existing_col(df, ZIP_COL_CANDIDATES)

    if muni_col:
        tmp = pd.DataFrame({"市区町村": normalize_muni_name(df[muni_col])})
    elif zip_col:
        z = normalize_zip(df[zip_col])
        tmp = pd.DataFrame({"市区町村": z.map(zip_to_muni)})
    else:
        raise KeyError(f"{Path(p).name}: 市区町村列も郵便番号列もありません。columns={list(df.columns)[:30]}")

    tmp = tmp[tmp["市区町村"].notna() & (tmp["市区町村"] != "")]
    visitors = tmp.groupby("市区町村", as_index=False).size().rename(columns={"size": "来場者数"})

    merged = visitors.merge(pop_df, on="市区町村", how="left")
    merged["集客率(%)"] = (merged["来場者数"] / merged["人口"]) * 100
    merged["集客率(1万人あたり)"] = (merged["来場者数"] / merged["人口"]) * 10000
    merged = merged.sort_values("集客率(1万人あたり)", ascending=False)

    venue_detail_tables[venue] = merged

    valid = merged["集客率(1万人あたり)"].replace([np.inf, -np.inf], np.nan).dropna()
    records.append({
        "会場": venue,
        "対象CSV": Path(p).name,
        "市区町村数": int(len(merged)),
        "総来場者数": int(merged["来場者数"].sum()),
        "平均集客率(1万人あたり)": float(valid.mean()) if len(valid) else np.nan,
        "中央値集客率(1万人あたり)": float(valid.median()) if len(valid) else np.nan,
        "最大集客率(1万人あたり)": float(valid.max()) if len(valid) else np.nan,
        "最小集客率(1万人あたり)": float(valid.min()) if len(valid) else np.nan,
    })

venue_summary = pd.DataFrame(records).sort_values("会場")
venue_summary.to_csv(OUT_DIR / "会場別サマリー.csv", index=False, encoding="utf-8-sig")

with pd.ExcelWriter(OUT_DIR / "会場別_市区町村明細.xlsx", engine="openpyxl") as writer:
    for venue, table in venue_detail_tables.items():
        table.to_excel(writer, index=False, sheet_name=(venue[:31] if venue else "venue"))

base = venue_summary["平均集客率(1万人あたり)"]
stats_df = pd.DataFrame([{
    "会場数": int(len(base)),
    "平均": float(base.mean()),
    "中央値": float(base.median()),
    "標準偏差": float(base.std(ddof=1)),
    "最小": float(base.min()),
    "最大": float(base.max()),
    "25%点": float(base.quantile(0.25)),
    "75%点": float(base.quantile(0.75)),
}])
stats_df.to_csv(OUT_DIR / "全会場_統計量.csv", index=False, encoding="utf-8-sig")

venue_summary.head(), stats_df


In [ ]:
# ===== 会場別マップ出力 =====
with open(GEOJSON_PATH, "r", encoding="utf-8") as f:
    gj = json.load(f)

prop_candidates = ["N03_004", "市区町村", "name", "NAME"]

def geojson_muni_name(feat):
    props = feat.get("properties", {})
    for c in prop_candidates:
        if c in props and props[c]:
            return str(props[c])
    return None

for venue, detail in venue_detail_tables.items():
    rate_map = dict(zip(detail["市区町村"], detail["集客率(1万人あたり)"]))
    m = folium.Map(location=[35.68, 139.76], zoom_start=5, tiles="cartodbpositron")

    vals = pd.Series(list(rate_map.values())).replace([np.inf, -np.inf], np.nan).dropna()
    vmin, vmax = (vals.min(), vals.max()) if len(vals) else (0, 1)
    span = (vmax - vmin) if vmax > vmin else 1.0

    def style_fn(feature):
        muni = normalize_muni_name(pd.Series([geojson_muni_name(feature)])).iloc[0]
        v = rate_map.get(muni, np.nan)
        if pd.isna(v):
            return {"fillColor": "#dddddd", "color": "#999999", "weight": 0.4, "fillOpacity": 0.2}
        t = (v - vmin) / span
        color = f"#{int(255*t):02x}40{int(255*(1-t)):02x}"
        return {"fillColor": color, "color": "#666666", "weight": 0.4, "fillOpacity": 0.7}

    folium.GeoJson(gj, style_function=style_fn).add_to(m)
    m.save(OUT_DIR / f"map_{venue}.html")

print(f"出力完了: {OUT_DIR.resolve()}")
